# Day 82
Given the Chargers blowing a 27 point lead to the Jaguars in yesterday's playoff game, I wanted to look at other instances of teams that came back to win after trailing at any point in the game.

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette('deep')

# Create database connection
conn = sqlite3.connect('../../data/db/database_test.db')

In [3]:
query = """
WITH home_games AS (
    SELECT
        game_id,
        season,
        week,
        season_type,
        home_team AS team,
        MAX(total_away_score - total_home_score) AS point_diff
    FROM pbp
    WHERE home_score > away_score -- home team won
        AND total_home_score < total_away_score -- score only when trailing
    GROUP BY season, week, team
), away_games AS (
    SELECT
        game_id,
        season,
        week,
        season_type,
        away_team AS team,
        MAX(total_home_score - total_away_score) AS point_diff
    FROM pbp
    WHERE away_score > home_score -- away team won
        AND total_away_score < total_home_score -- score only when trailing
    GROUP BY season, week, team
)
SELECT *
FROM home_games
UNION ALL
SELECT *
FROM away_games
ORDER BY point_diff DESC
;
"""

df = pd.read_sql(query, conn)
df.head()

,game_id,season,week,season_type,team,point_diff
0,2022_15_IND_MIN,2022,15,REG,MIN,33.0
1,2013_18_KC_IND,2013,18,POST,IND,28.0
2,2014_05_CLE_TEN,2014,5,REG,CLE,25.0
3,2016_21_NE_ATL,2016,21,POST,NE,25.0
4,2002_18_NYG_SF,2002,18,POST,SF,24.0


## Regular Season

In [8]:
df.query("season_type == 'REG'").head(10)

,game_id,season,week,season_type,team,point_diff
0,2022_15_IND_MIN,2022,15,REG,MIN,33.0
2,2014_05_CLE_TEN,2014,5,REG,CLE,25.0
5,2013_12_DEN_NE,2013,12,REG,NE,24.0
6,2015_07_TB_WAS,2015,7,REG,WAS,24.0
8,2011_04_DET_DAL,2011,4,REG,DET,24.0
9,2012_06_DEN_SD,2012,6,REG,DEN,24.0
10,2000_08_MIA_NYJ,2000,8,REG,NYJ,23.0
11,2013_15_GB_DAL,2013,15,REG,GB,23.0
12,1999_02_IND_NE,1999,2,REG,NE,21.0
13,1999_04_CAR_WAS,1999,4,REG,WAS,21.0


## Postseason

In [9]:
df.query("season_type == 'POST'").head(10)

,game_id,season,week,season_type,team,point_diff
1,2013_18_KC_IND,2013,18,POST,IND,28.0
3,2016_21_NE_ATL,2016,21,POST,NE,25.0
4,2002_18_NYG_SF,2002,18,POST,SF,24.0
7,2019_19_HOU_KC,2019,19,POST,KC,24.0
57,2006_20_NE_IND,2006,20,POST,IND,18.0
69,2017_18_TEN_KC,2017,18,POST,TEN,18.0
71,2021_21_CIN_KC,2021,21,POST,CIN,18.0
80,2002_18_CLE_PIT,2002,18,POST,PIT,17.0
119,2012_20_SF_ATL,2012,20,POST,SF,17.0
137,2014_20_GB_SEA,2014,20,POST,SEA,16.0


Postseason has less variance which makes sense given the smaller sample size but I think it's mainly because these are the best teams so games will tend to be closer.  

The Jaguar's 27 point comeback is the 2nd highest in postseason history since 1999.